In [1]:
from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True)
print(X[:5])
print(y[:5])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
[0 0 0 0 0]


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=10, test_size=0.2, stratify=y, shuffle=True
)
print(len(X_train))

120


In [5]:
from torch import nn
import torch.nn.functional as F


# 신경망 모형
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()

        # input layer, Linear 선형함수(1차함수)
        # input nodes, output nodes 50
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 20)
        self.layer3 = nn.Linear(20, 3)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))

        # 출력층의 활성화함수 - 소프트맥스
        # dim=0 첫번째 차원(row), row 단위로 softmax 적용
        x = F.softmax(self.layer3(x), dim=0)
        return x

In [6]:
import torch

model = Model(X_train.shape[1])  # 초기화함수의 input_dim으로 변수개수가 전달됨
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # 최적화함수 정의
loss_fn = nn.CrossEntropyLoss()  # 손실함수 정의
epochs = 100

In [7]:
# 넘파이배열로부터 텐서를 만들고
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()

for epoch in range(1, epochs + 1):
    print("Epoch", epoch)
    y_pred = model(X_train)  # 예측값

    # 손실함수에 예측값과 실제값을 입력
    loss = loss_fn(y_pred, y_train)
    print("loss: ", loss.item())

    # 경사 초기화
    optimizer.zero_grad()
    loss.backward()  # 역전파
    optimizer.step()  # 가중치 업데이트

Epoch 1
loss:  1.0981792211532593
Epoch 2
loss:  1.096538782119751
Epoch 3
loss:  1.094946026802063
Epoch 4
loss:  1.0935866832733154
Epoch 5
loss:  1.0921117067337036
Epoch 6
loss:  1.0907256603240967
Epoch 7
loss:  1.0897377729415894
Epoch 8
loss:  1.0889866352081299
Epoch 9
loss:  1.0883723497390747
Epoch 10
loss:  1.0878562927246094
Epoch 11
loss:  1.0874507427215576
Epoch 12
loss:  1.0871467590332031
Epoch 13
loss:  1.086926817893982
Epoch 14
loss:  1.0867507457733154
Epoch 15
loss:  1.0865988731384277
Epoch 16
loss:  1.086423635482788
Epoch 17
loss:  1.0862585306167603
Epoch 18
loss:  1.0861093997955322
Epoch 19
loss:  1.0859408378601074
Epoch 20
loss:  1.0857207775115967
Epoch 21
loss:  1.0854454040527344
Epoch 22
loss:  1.0851547718048096
Epoch 23
loss:  1.084939956665039
Epoch 24
loss:  1.0847464799880981
Epoch 25
loss:  1.0845249891281128
Epoch 26
loss:  1.0843054056167603
Epoch 27
loss:  1.08408522605896
Epoch 28
loss:  1.0838702917099
Epoch 29
loss:  1.0836611986160278
Epoc

In [8]:
X_test = torch.from_numpy(X_test).float()
pred = model(X_test)
pred[:5]

tensor([[7.2103e-08, 7.6628e-03, 5.8448e-05],
        [1.6766e-02, 1.8336e-06, 1.3533e-11],
        [2.4254e-08, 1.8643e-03, 1.6470e-04],
        [4.7400e-09, 5.3889e-04, 6.8121e-04],
        [6.4397e-06, 3.3602e-01, 6.8676e-07]], grad_fn=<SliceBackward0>)

In [9]:
import numpy as np

np.argmax(pred.data.numpy(), axis=1)

array([1, 0, 1, 2, 1, 2, 0, 2, 2, 0, 0, 1, 2, 2, 1, 0, 0, 1, 2, 0, 2, 2,
       2, 0, 0, 1, 1, 0, 1, 1], dtype=int64)

In [10]:
from sklearn.metrics import accuracy_score

# 모형의 정확도 측정
accuracy_score(y_test, np.argmax(pred.data.numpy(), axis=1))

1.0

In [11]:
torch.save(model, "c:/data/iris/iris-torch.h5")

In [12]:
model2 = torch.load("c:/data/iris/iris-torch.h5")

np.argmax(model2(X_test[0]).data.numpy())

2

In [14]:
%pip install torchinfo
from torchinfo import summary

summary(model)
summary(model, input_size=(32, 4))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [32, 3]                   --
├─Linear: 1-1                            [32, 50]                  250
├─Linear: 1-2                            [32, 20]                  1,020
├─Linear: 1-3                            [32, 3]                   63
Total params: 1,333
Trainable params: 1,333
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.04
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.01
Estimated Total Size (MB): 0.02